# Fine-tuning SmolVLM using direct preference optimization (DPO) with TRL on a consumer GPU

_Authored by: [Sergio Paniego](https://github.com/sergiopaniego)_


In this recipe, we’ll demonstrate how to fine-tune a smol 🤏 Vision Language Model (VLM) using the Hugging Face ecosystem using direct preference optimization (DPO), leveraging the powerful Transformer Reinforcement Learning library (TRL). This step-by-step guide will enable you to customize VLMs for your specific tasks, even on consumer GPUs.


We'll fine-tune SmolVLM using a preference dataset. If you're new to Preference Optimization for LLM/VLM, you can get a deeper understanding about it in [this blog](https://huggingface.co/blog/dpo_vlm). The preference dataset is [HuggingFaceH4/rlaif-v_formatted](https://huggingface.co/datasets/HuggingFaceH4/rlaif-v_formatted) which includes pairs of prompt+image and a chosen a rejected answer for each of them. The goal of our training is to make the model more prone to the chosen decisions of the preference dataset.

## 📖 Additional Resources

Expand your knowledge of Vision Language Models and related tools with these resources:

- **[Multimodal Recipes in Cookbook](https://huggingface.co/learn/cookbook/index):** Explore practical recipes for multimodal models, including RAG pipelines and fine-tuning. We already have [a recipe for fine-tuning a smol VLM with TRL](https://huggingface.co/learn/cookbook/fine_tuning_smol_vlm_sft_trl), so refer to it for more details.
- **[TRL Community Tutorials](https://huggingface.co/docs/trl/main/en/community_tutorials):** A treasure trove of tutorials to deepen your understanding of TRL and its applications.

With these resources, you’ll be equipped to dive deeper into the world of VLMs and push the boundaries of what they can achieve!

This notebook is tested using a L4 GPU.


![Smol VLMs comparison](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/smolvlm_ecosystem.png)

# 1. Install Dependencies

Let’s start by installing the essential libraries we’ll need for fine-tuning! 🚀

In [ ]:
!pip install  -U -q transformers trl datasets bitsandbytes peft accelerate
# Tested with transformers==4.46.3, trl==0.12.2, datasets==3.2.0, bitsandbytes==0.45.0, peft==0.14.0, accelerate==1.2.0

In [3]:
!pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Authenticate with your Hugging Face account to save and share your model directly from this notebook 🗝️.

In [4]:
from huggingface_hub import notebook_login

notebook_login()

# 2. Load Dataset 📁

We’ll load the [HuggingFaceH4/rlaif-v_formatted](https://huggingface.co/datasets/HuggingFaceH4/rlaif-v_formatted) dataset, which includes pairs of prompt+image and a chosen a rejected answer for each of them.

The dataset we'll use is already formatted using this format, otherwise you may need to format it.

In [5]:
from datasets import load_dataset

dataset_id = "HuggingFaceH4/rlaif-v_formatted"
train_dataset, test_dataset = load_dataset(dataset_id, split=['train[:5%]', 'test[:1%]'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

train-00000-of-00013.parquet:   0%|          | 0.00/569M [00:00<?, ?B/s]

train-00001-of-00013.parquet:   0%|          | 0.00/473M [00:00<?, ?B/s]

train-00002-of-00013.parquet:   0%|          | 0.00/448M [00:00<?, ?B/s]

...

train-00011-of-00013.parquet:   0%|          | 0.00/476M [00:00<?, ?B/s]

train-00012-of-00013.parquet:   0%|          | 0.00/510M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/399M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78975 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4157 [00:00<?, ? examples/s]

We will ensure all the images are RGB formatted:

In [6]:
from PIL import Image

def ensure_rgb(example):
    # Convert the image to RGB if it's not already
    image = example['images'][0]
    if isinstance(image, Image.Image):
        if image.mode != 'RGB':
            image = image.convert('RGB')
        example['images'] = [image]
    return example

# Apply the transformation to the dataset
train_dataset = train_dataset.map(ensure_rgb, num_proc=32)
test_dataset = test_dataset.map(ensure_rgb, num_proc=32)

Map (num_proc=32):   0%|          | 0/3949 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/42 [00:00<?, ? examples/s]

# 3. Fine-Tune the Model using TRL

## 3.1 Load the Quantized Model for Training ⚙️


Let's first load the model and processor, along with the quantized configuration using BitsAndBytes

In [7]:
import torch
from transformers import Idefics3ForConditionalGeneration, AutoProcessor

model_id = "HuggingFaceTB/SmolVLM-Instruct"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
from transformers import BitsAndBytesConfig

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = Idefics3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    _attn_implementation="flash_attention_2",
)
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


## 3.2 Set Up QLoRA and DPOConfig 🚀

Next, we’ll configure [QLoRA](https://github.com/artidoro/qlora) for our training setup. QLoRA allows efficient fine-tuning of large models by reducing the memory footprint. Unlike traditional LoRA, which uses low-rank approximation, QLoRA further quantizes the LoRA adapter weights, leading to even lower memory usage and faster training.

To boost efficiency, we can also leverage a **paged optimizer** or **8-bit optimizer** during QLoRA implementation. This approach enhances memory efficiency and speeds up computations, making it ideal for optimizing our model without sacrificing performance.

In [9]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
    use_dora=True,
    init_lora_weights="gaussian"
)

# Apply PEFT model adaptation
peft_model = get_peft_model(model, peft_config)

# Print trainable parameters
peft_model.print_trainable_parameters()

trainable params: 11,269,248 || all params: 2,257,542,128 || trainable%: 0.4992


In [12]:
from trl import DPOConfig

'''
# A100
training_args = DPOConfig(
    output_dir="smolvlm-instruct-trl-dpo-rlaif-v",
    bf16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=32,
    num_train_epochs=5,
    dataset_num_proc=32,  # tokenization will use 32 processes
    dataloader_num_workers=32,  # data loading will use 32 workers
    logging_steps=10,
    report_to="tensorboard",
    push_to_hub=True,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=1,
    eval_steps=10,  # Steps interval for evaluation
    eval_strategy="steps",
)
'''

training_args = DPOConfig(
    output_dir="t4-smolvlm-instruct-trl-dpo-rlaif-v",
    bf16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=32,
    num_train_epochs=5,
    dataset_num_proc=8,  # tokenization will use 32 processes
    dataloader_num_workers=8,  # data loading will use 32 workers
    logging_steps=10,
    report_to="tensorboard",
    push_to_hub=True,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=1,
    eval_steps=10,  # Steps interval for evaluation
    eval_strategy="steps",
)

We will use Direct Preference Optimization (DPO) to improve our model's performance on the specific task. To achieve this, we'll define the training arguments with the [DPOTrainer](https://huggingface.co/docs/trl/dpo_trainer) class from the [TRL library](https://huggingface.co/docs/trl/index). DPO leverages labeled data to help the model generate prefered responses.

In [ ]:
from trl import DPOTrainer

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    tokenizer=processor,
)

Extracting prompt from train dataset (num_proc=8):   0%|          | 0/3949 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=8):   0%|          | 0/3949 [00:00<?, ? examples/s]

Extracting prompt from eval dataset (num_proc=8):   0%|          | 0/42 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=8):   0%|          | 0/42 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/3949 [00:00<?, ? examples/s]

Time to Train the Model! 🎉

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
10,0.694500,0.687105,-0.089659,-0.105017,0.568182,0.015358,-529.369629,-553.168213,-0.505493,-0.510487
20,0.692100,0.678315,-0.048025,-0.078889,0.522727,0.030864,-528.953308,-552.906982,-0.505605,-0.510141
30,0.693700,0.688169,-0.128291,-0.145414,0.590909,0.017123,-529.755981,-553.572266,-0.518147,-0.521626
40,0.685000,0.688010,-0.162893,-0.176594,0.568182,0.013701,-530.101990,-553.884033,-0.529034,-0.533426
50,0.686500,0.682640,-0.101526,-0.128467,0.431818,0.026941,-529.488281,-553.402771,-0.507708,-0.509508
60,0.687100,0.687270,-0.119737,-0.134479,0.522727,0.014742,-529.670410,-553.462830,-0.511138,-0.514428
70,0.686200,0.682125,-0.064500,-0.090859,0.545455,0.026359,-529.118103,-553.026733,-0.499416,-0.504765
80,0.684100,0.684057,-0.022482,-0.041385,0.522727,0.018903,-528.697815,-552.531921,-0.479619,-0.483792
90,0.680800,0.679046,-0.078754,-0.109563,0.613636,0.030809,-529.260559,-553.213745,-0.497246,-0.503218
100,0.680000,0.687835,-0.104676,-0.117668,0.545455,0.012991,-529.519836,-553.294861,-0.507647,-0.512477


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, wh

TrainOutput(global_step=150, training_loss=0.6847708098093669, metrics={'train_runtime': 19894.0697, 'train_samples_per_second': 0.993, 'train_steps_per_second': 0.008, 'total_flos': 0.0, 'train_loss': 0.6847708098093669, 'epoch': 4.919028340080971})

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations

Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
10,0.691500,0.678604,-0.101302,-0.131777,0.522727,0.030475,-529.486084,-553.435791,-0.512805,-0.516536
20,0.692300,0.684393,-0.110511,-0.130010,0.659091,0.019499,-529.578186,-553.418213,-0.510626,-0.517021
30,0.694600,0.673025,-0.108794,-0.151864,0.590909,0.043070,-529.560974,-553.636780,-0.513864,-0.516869


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, wh

TrainOutput(global_step=30, training_loss=0.6928022384643555, metrics={'train_runtime': 3969.4356, 'train_samples_per_second': 0.995, 'train_steps_per_second': 0.008, 'total_flos': 0.0, 'train_loss': 0.6928022384643555, 'epoch': 0.97165991902834})

Let's save the results 💾

In [ ]:
trainer.save_model(training_args.output_dir)